In [33]:
import xarray as xr
import numpy as np
import glob
import csv                  # to save indices
import pandas as pd

# Check whether assymtry in events exists in wind forcing on Labrador Sea

In [21]:
path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/comp/all_member_timeseries/'

wind_files = sorted(glob.glob(path + 'monthly_tau_member*'))


In [42]:
window_size = 40
duration_threshold = 20
period = window_size + duration_threshold

threshold_multiple = 1

criteria_indices_dict = {}

In [47]:
for i in range(len(wind_files)):
    
    ds = xr.open_dataarray(wind_files[i])
    
    signal = ds.resample(time='1Y').mean().sel(time=slice(None, '2015'))
    criteria_indices = []
    for j in range(window_size, len(signal) - duration_threshold):

        # Compute mean and std of period 1
        window = signal[j - window_size:j]
        std_window = np.std(window)
        avg_window = np.mean(window)

        # compute mean of period 2
        threshold = signal[j:j + duration_threshold]
        avg_threshold = np.mean(threshold)

        # Check if the std is above the threshold
        if avg_threshold >= avg_window + (threshold_multiple * std_window):
            condition = "Above"
        elif avg_threshold <= avg_window - (threshold_multiple * std_window):
            condition = "Below"
        else:
            condition = "Within"

        if condition != "Within":
            if len(criteria_indices) == 0:
                criteria_indices.append((j, condition))
            elif j >= criteria_indices[-1][0] + 10:
                criteria_indices.append((j, condition))
                
            #print('event found')

    # Save criteria_indices in the dictionary
    criteria_indices_dict[i] = criteria_indices

In [48]:
# handle data in dictionary
result = []
for index, values in criteria_indices_dict.items():
    if values:
        result.append((index, values))

In [49]:
# save data to csv file
with open('1_std_events_wind_lab.csv', 'w', newline='') as csvfile:
    fieldnames = ['Index', 'Values', 'Condition']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index, values in result:
        for value, condition in values:
            writer.writerow({'Index': index, 'Values': value, 'Condition': condition})

In [50]:
# perfrom pandas anaylsis
df = pd.read_csv('1_std_events_wind_lab.csv')
grouped = df.groupby('Index')
condition_counts = df['Condition'].value_counts()
print(condition_counts)

Above    19
Name: Condition, dtype: int64
